In [ ]:
import pandas as pd
from google.colab import files
import os

# Step 1: Upload all 11 raw Excel files
print("Upload ALL 11 raw files now (Jan to Dec 2015.xlsx to Jan to Dec 2025.xlsx):")
uploaded = files.upload()

# Get sorted list of files by year
arrival_files = sorted([f for f in uploaded.keys() if f.startswith('Jan to Dec') and f.endswith('.xlsx')],
                       key=lambda x: int(x.split(' ')[-1].split('.')[0]))

print(f"\nUploaded {len(arrival_files)} files:")
for f in arrival_files:
    print(f"  • {f}")

Upload ALL 11 raw files now (Jan to Dec 2015.xlsx to Jan to Dec 2025.xlsx):


Saving Jan to Dec 2015.xlsx to Jan to Dec 2015.xlsx
Saving Jan to Dec 2016.xlsx to Jan to Dec 2016.xlsx
Saving Jan to Dec 2017.xlsx to Jan to Dec 2017.xlsx
Saving Jan to Dec 2018.xlsx to Jan to Dec 2018.xlsx
Saving Jan to Dec 2019.xlsx to Jan to Dec 2019.xlsx
Saving Jan to Dec 2020.xlsx to Jan to Dec 2020.xlsx
Saving Jan to Dec 2021.xlsx to Jan to Dec 2021.xlsx
Saving Jan to Dec 2022.xlsx to Jan to Dec 2022.xlsx
Saving Jan to Dec 2023.xlsx to Jan to Dec 2023.xlsx
Saving Jan to Dec 2024.xlsx to Jan to Dec 2024.xlsx
Saving Jan to Dec 2025.xlsx to Jan to Dec 2025.xlsx

Uploaded 11 files:
  • Jan to Dec 2015.xlsx
  • Jan to Dec 2016.xlsx
  • Jan to Dec 2017.xlsx
  • Jan to Dec 2018.xlsx
  • Jan to Dec 2019.xlsx
  • Jan to Dec 2020.xlsx
  • Jan to Dec 2021.xlsx
  • Jan to Dec 2022.xlsx
  • Jan to Dec 2023.xlsx
  • Jan to Dec 2024.xlsx
  • Jan to Dec 2025.xlsx


In [ ]:
# Step 2: Function to process each yearly file without dropping any rows or data
def process_yearly_file(file_name, year):
    # Read the Excel file, skipping no rows (handle varying headers)
    df_year = pd.read_excel(file_name, header=0)  # Assume first row is header

    # Drop any 'No', 'Rank', or unnamed index columns if present, but keep all data rows
    cols_to_drop = [col for col in df_year.columns if str(col).lower().startswith('no') or str(col).lower().startswith('rank') or 'unnamed' in str(col).lower()]
    df_year = df_year.drop(columns=cols_to_drop, errors='ignore')

    # Assume first column is 'Country', remaining are months
    columns = df_year.columns.tolist()
    country_col = columns[0]
    month_cols = columns[1:]

    # Standardize month names to abbr (e.g., 'January' -> 'Jan', 'Jan' remains 'Jan')
    month_map = {
        'January': 'Jan', 'February': 'Feb', 'March': 'Mar', 'April': 'Apr', 'May': 'May', 'June': 'Jun',
        'July': 'Jul', 'August': 'Aug', 'September': 'Sep', 'October': 'Oct', 'November': 'Nov', 'December': 'Dec',
        'Jan': 'Jan', 'Feb': 'Feb', 'Mar': 'Mar', 'Apr': 'Apr', 'May': 'May', 'Jun': 'Jun',
        'Jul': 'Jul', 'Aug': 'Aug', 'Sep': 'Sep', 'Oct': 'Oct', 'Nov': 'Nov', 'Dec': 'Dec'
    }
    # Clean month names (strip spaces, capitalize)
    standardized_months = [month_map.get(col.strip().capitalize(), col) for col in month_cols]

    # Set columns
    df_year.columns = [country_col] + standardized_months

    # Melt to long format: Keep ALL rows, even if empty or NaN
    df_melt = pd.melt(df_year, id_vars=[country_col], value_vars=standardized_months,
                      var_name='Month', value_name='Arrivals')

    # Rename Country column
    df_melt = df_melt.rename(columns={country_col: 'Country'})

    # Create Month_Year in 'Year-Month' format (e.g., '2025-Jan')
    df_melt['Month_Year'] = f"{year}-" + df_melt['Month']

    # Convert Arrivals to numeric, coerce errors to 0 (no dropna)
    df_melt['Arrivals'] = pd.to_numeric(df_melt['Arrivals'], errors='coerce').fillna(0)

    # Standardize Country names (upper, strip, basic mappings - no dropping)
    df_melt['Country'] = df_melt['Country'].astype(str).str.upper().str.strip()
    country_mappings = {
        'VIET NAM': 'VIETNAM',
        'YEMEN (YEMEN ARAB REP.)': 'YEMEN',
        'YEMEN (YEMEN ARAB REPUBLIC)': 'YEMEN',
        'ZAMBIA (NORTHERN RHODESIA)': 'ZAMBIA',
        'ZAMBIA (NORTHERNRHODESIA)': 'ZAMBIA',
        'ZAMBIA(NORTHERN RHODESIA)': 'ZAMBIA',
        'ANTIGUA AND BARBUDA': 'ANTIGUA & BARBUDA',
        'YEMEN': 'YEMEN',
        # Add any other inconsistent names from your files
    }
    df_melt['Country'] = df_melt['Country'].replace(country_mappings)

    # Select only required columns, no filtering
    return df_melt[['Country', 'Month_Year', 'Arrivals']]

In [ ]:
# Step 3: Process all files and concatenate
all_dfs = []
for file_name in arrival_files:
    year = int(file_name.split(' ')[-1].split('.')[0])
    df_year = process_yearly_file(file_name, year)
    all_dfs.append(df_year)

# Concatenate all - no grouping or summing here to avoid any loss
merged_df = pd.concat(all_dfs, ignore_index=True)

In [ ]:
# Step 4: Verify totals match your raw sums (compute yearly sums)
yearly_totals = merged_df.groupby(merged_df['Month_Year'].str[:4])['Arrivals'].sum().reset_index()
yearly_totals.columns = ['Year', 'Total Arrivals']
print("\nYearly Totals (should match your raw sums):")
print(yearly_totals)

grand_total = merged_df['Arrivals'].sum()
print(f"\nGrand Total 2015-2025: {grand_total}")



Yearly Totals (should match your raw sums):
    Year  Total Arrivals
0   2015       1614546.0
1   2016       1839960.0
2   2017       1929456.0
3   2018       2138214.0
4   2019       1736718.0
5   2020        507704.0
6   2021        134018.0
7   2022        660219.0
8   2023       1335807.0
9   2024       1869307.0
10  2025       2149615.0

Grand Total 2015-2025: 15915564.0


In [ ]:
# Step 5: Save the final cleaned file
output_file = 'ArrivalDataFinal2015-2025.xlsx'
merged_df.to_excel(output_file, index=False)
files.download(output_file)

print(f"\nCell complete – Saved '{output_file}' with {len(merged_df)} rows (no data loss). Use this for trends/forecasts.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Cell complete – Saved 'ArrivalDataFinal2015-2025.xlsx' with 19448 rows (no data loss). Use this for trends/forecasts.
